In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# 노트북 안에 그래프 그리기
%matplotlib inline

# ggplot: 그래프에서 격자로 숫자 범위 눈에 잘 띄도록 하는 스타일
plt.style.use('ggplot')

# 그래프에서 마이넛 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/KU/2024-2/데과응/중간과제

/content/drive/MyDrive/KU/2024-2/데과응/중간과제


## Clean Train Data

In [ ]:
df = pd.read_csv("./question1-train.csv", parse_dates=["datetime"])
print(len(df))
df.head(20)

9025


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14


In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df["dayofweek"] = df["datetime"].dt.dayofweek
print(df.shape)
df.head()

(9025, 16)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,4,5


In [ ]:
# outlier 제거
df = df[np.abs(df['count'] - df['count'].mean()) <= (3 * df['count'].std())]
print(df.shape)

(8893, 16)


In [ ]:
def predict_windspeed(data):
    # windspeed==0 과 !=0 나눠주기
    df_windspeed_0 = data.loc[df['windspeed'] == 0] # test dataset
    df_windspeed_not_0 = data.loc[df['windspeed'] != 0] # train dataset

    # windspeed 예측 feature
    wCol = ["season", "month", "hour", "humidity"]

    df_windspeed_not_0["windspeed"] = df_windspeed_not_0["windspeed"].astype("str")

    rfModel_windspeed = RandomForestClassifier()

    # wCol features 바탕으로 rfModel_windspeed 학습
    rfModel_windspeed.fit(df_windspeed_not_0[wCol], df_windspeed_not_0["windspeed"])

    # prediction
    windspeed_0_pred = rfModel_windspeed.predict(X = df_windspeed_0[wCol])

    # new dataframe
    predict_windspeed_0 = df_windspeed_0
    predict_windspeed_not_0 = df_windspeed_not_0

    # 값이 0인 windspeed 예측값으로 대체
    predict_windspeed_0["windspeed"] = windspeed_0_pred

    # df_windspeed_not_0에서 0이 아닌 windspeed값이 있는 dataframe + 예측값 있는 dataframe
    # data = predict_windspeed_not_0.append(predict_windspeed_0)   <- .append() 더 이상 pandas에서 지원X
    data = pd.concat([predict_windspeed_not_0, predict_windspeed_0], ignore_index=True)


    # datatype str -> float
    data["windspeed"] = data["windspeed"].astype("float")

    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)

    return data

In [ ]:
df = predict_windspeed(df)

<ipython-input-7-6fdcdfd4a3ab>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_windspeed_not_0["windspeed"] = df_windspeed_not_0["windspeed"].astype("str")
<ipython-input-7-6fdcdfd4a3ab>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_windspeed_0["windspeed"] = windspeed_0_pred


In [ ]:
df = df[df['weather'] != 4]

print(df.shape)
df.head()

(8892, 16)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,dayofweek
0,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1,2011,1,5,5
1,2011-01-01 10:00:00,1,0,0,1,15.58,19.695,76,16.9979,12,24,36,2011,1,10,5
2,2011-01-01 11:00:00,1,0,0,1,14.76,16.665,81,19.0012,26,30,56,2011,1,11,5
3,2011-01-01 12:00:00,1,0,0,1,17.22,21.210,77,19.0012,29,55,84,2011,1,12,5
4,2011-01-01 13:00:00,1,0,0,2,18.86,22.725,72,19.9995,47,47,94,2011,1,13,5


In [ ]:
drop_features = ['casual', 'registered']

df = df.drop(drop_features, axis=1)

print(df.shape)
df.head()

(8892, 14)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,hour,dayofweek
0,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,1,2011,1,5,5
1,2011-01-01 10:00:00,1,0,0,1,15.58,19.695,76,16.9979,36,2011,1,10,5
2,2011-01-01 11:00:00,1,0,0,1,14.76,16.665,81,19.0012,56,2011,1,11,5
3,2011-01-01 12:00:00,1,0,0,1,17.22,21.210,77,19.0012,84,2011,1,12,5
4,2011-01-01 13:00:00,1,0,0,2,18.86,22.725,72,19.9995,94,2011,1,13,5


In [ ]:
df.to_csv('question1-train_cleaned.csv', index=False)

## Clean Test Data

In [ ]:
df_test = pd.read_excel('./question1-test - features.xlsx')

print(df_test.shape)
df_test.head()

(1861, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,NaN,NaN
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,NaN,NaN
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,NaN,NaN
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,NaN,NaN
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,NaN,NaN


In [ ]:
df_test.isnull().sum()

,0
datetime,0
season,0
holiday,0
workingday,0
weather,0
temp,0
atemp,0
humidity,0
windspeed,0
casual,1861


In [ ]:
df_test['year'] = df_test['datetime'].dt.year
df_test['month'] = df_test['datetime'].dt.month
df_test['hour'] = df_test['datetime'].dt.hour
df_test["dayofweek"] = df_test["datetime"].dt.dayofweek
print(df_test.shape)
df_test.head()

(1861, 16)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,dayofweek
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,NaN,NaN,2012,4,13,0
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,NaN,NaN,2012,4,14,0
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,NaN,NaN,2012,4,15,0
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,NaN,NaN,2012,4,16,0
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,NaN,NaN,2012,4,17,0


In [ ]:
df_test = df_test.drop(drop_features, axis=1)

print(df_test.shape)
df_test.head()

(1861, 14)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,hour,dayofweek
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,2012,4,13,0
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,2012,4,14,0
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,2012,4,15,0
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,2012,4,16,0
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,2012,4,17,0


In [ ]:
df_test.to_csv('question1-test_cleaned.csv', index=False)

## Model (XGBoost)

In [ ]:
categorical_col = ['workingday', 'weather','dayofweek','month','year','hour']
numerical_col = ['temp', 'humidity']

target_col = "count"

X = df[categorical_col + numerical_col]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1203.6007119753126


### Trial 2
kaggle notebook parameter https://www.kaggle.com/code/jaeraklim/bike-sharing-demand

In [ ]:
best_params = {
      'n_estimators': 973,
      'max_depth': 5,
      'learning_rate': 0.0276061670314364,
      'subsample': 0.7326901697354489,
      'colsample_bytree': 0.7810298851531396,
      'gamma': 0.06562510648004544,
      'reg_alpha': 0.0019110789512788793,
      'reg_lambda': 3.923321423779548e-06
  }

xgb_bestparam = XGBRegressor(**best_params)
xgb_bestparam.fit(X_train, y_train)
y_pred = xgb_bestparam.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1108.1676359576327


### Trials 3
GraphSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
        'n_estimators': [100, 300, 500, 1000],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'min_child_weight': [1, 3, 5],
        'gamma': [0, 0.1, 0.2]
    }

model = XGBRegressor(random_state=42)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)


Fitting 5 folds for each of 3888 candidates, totalling 19440 fits
Best Parameters: {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 1000, 'subsample': 0.8}


In [ ]:
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1010.5405625864414


In [ ]:
from sklearn.model_selection import KFold
import numpy as np

def kfold_xgboost(X, y, best_params):
    # KFold 초기화
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # MSE 저장을 위한 리스트
    mse_scores = []

    # 최적 파라미터로 모델 초기화
    best_params = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 1000, 'subsample': 0.8}
    model = XGBRegressor(**best_params, random_state=42)

    # K-fold 교차 검증 수행
    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        # 학습 데이터와 검증 데이터 분할
        X_train_fold = X.iloc[train_index]
        y_train_fold = y.iloc[train_index]
        X_val_fold = X.iloc[val_index]
        y_val_fold = y.iloc[val_index]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 예측 및 MSE 계산
        y_pred_fold = model.predict(X_val_fold)
        mse_fold = mean_squared_error(y_val_fold, y_pred_fold)
        mse_scores.append(mse_fold)

        print(f"Fold {fold} MSE: {mse_fold:.4f}")

    # 평균 MSE 계산
    mean_mse = np.mean(mse_scores)
    print(f"\nAverage MSE across all folds: {mean_mse:.4f}")

    return mean_mse

# 이전에 찾은 최적 파라미터로 K-fold 교차 검증 수행
average_mse = kfold_xgboost(X, y, best_params)

Fold 1 MSE: 1006.3449
Fold 2 MSE: 945.3873
Fold 3 MSE: 1069.1031
Fold 4 MSE: 1130.0779
Fold 5 MSE: 1077.7308

Average MSE across all folds: 1045.7288


## Model (GradientBoosting)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial, X_train, X_test, y_train, y_test):
    # 하이퍼파라미터 탐색 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),

        # 추가적인 파라미터들
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.2),
        'validation_fraction': trial.suggest_float('validation_fraction', 0.1, 0.3),
        'n_iter_no_change': trial.suggest_int('n_iter_no_change', 5, 20)
    }

    # 모델 생성
    gbm_model = GradientBoostingRegressor(
        **params,
        random_state=42
    )

    # 교차 검증 점수 계산
    scores = cross_val_score(
        gbm_model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=1
    )

    # MSE의 평균 반환
    return -scores.mean()

def gradient_boosting_optuna_search(X_train, X_test, y_train, y_test, n_trials=100):
    # Optuna study 생성
    study = optuna.create_study(direction='minimize')

    # 최적화 실행
    study.optimize(
        lambda trial: objective(trial, X_train, X_test, y_train, y_test),
        n_trials=n_trials,
        n_jobs=1,
        show_progress_bar=True
    )

    # 최적의 하이퍼파라미터 출력
    print("\nBest trial:")
    trial = study.best_trial
    print("  Value (Negative MSE):", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 최적의 모델로 최종 평가
    best_params = study.best_params
    best_model = GradientBoostingRegressor(**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # 테스트 세트에 대한 예측 및 MSE 계산
    y_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    print(f"\nTest MSE with best parameters: {test_mse:.4f}")

    # 학습 과정 시각화
    try:
        optuna.visualization.plot_optimization_history(study)
        optuna.visualization.plot_param_importances(study)
        optuna.visualization.plot_param_correlations(study)
    except (ImportError, ModuleNotFoundError):
        print("Plotly is not installed. Skipping visualizations.")

    return best_params, test_mse, study

# 실행
best_params, test_mse, study = gradient_boosting_optuna_search(
    X_train, X_test, y_train, y_test, n_trials=100
)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


KeyboardInterrupt: 

## Model (RandomForest)

In [ ]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial, X_train, X_test, y_train, y_test):
    # 하이퍼파라미터 탐색 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),  # 'auto' 제거
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # 모델 생성
    rf_model = RandomForestRegressor(**params, random_state=42)

    # 교차 검증 점수 계산
    scores = cross_val_score(
        rf_model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    # MSE의 평균 반환
    return -scores.mean()

def random_forest_optuna_search(X_train, X_test, y_train, y_test, n_trials=100):
    # Optuna study 생성
    study = optuna.create_study(direction='minimize')

    # 최적화 실행
    study.optimize(
        lambda trial: objective(trial, X_train, X_test, y_train, y_test),
        n_trials=n_trials,
        n_jobs=1  # 병렬 처리 관련 문제 방지를 위해 1로 설정
    )

    # 최적의 하이퍼파라미터 출력
    print("\nBest trial:")
    trial = study.best_trial
    print("  Value (Negative MSE):", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 최적의 모델로 최종 평가
    best_params = study.best_params
    best_model = RandomForestRegressor(**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # 테스트 세트에 대한 예측 및 MSE 계산
    y_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    print(f"\nTest MSE with best parameters: {test_mse:.4f}")

    # 학습 과정 시각화
    optuna.visualization.plot_optimization_history(study)
    optuna.visualization.plot_param_importances(study)

    return best_params, test_mse, study

# 실행
best_params, test_mse, study = random_forest_optuna_search(
    X_train, X_test, y_train, y_test, n_trials=100
)

## Model (CatBoost)

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective(trial, X_train, X_test, y_train, y_test):
    # 기본 하이퍼파라미터 탐색 공간 정의
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 7),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS', 'Bayesian']),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50)
    }

    # bootstrap_type에 따른 조건부 파라미터 설정
    if params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.8, 1.0)
    elif params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.0, 1.0)

    # 모델 생성
    cat_model = CatBoostRegressor(
        **params,
        random_seed=42,
        verbose=False,
        allow_writing_files=False
    )

    # 교차 검증 점수 계산
    scores = cross_val_score(
        cat_model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=1  # 병렬 처리 관련 문제 방지를 위해 1로 설정
    )

    return -scores.mean()

def catboost_optuna_search(X_train, X_test, y_train, y_test, n_trials=100):
    # Optuna study 생성
    study = optuna.create_study(direction='minimize')

    # 최적화 실행
    study.optimize(
        lambda trial: objective(trial, X_train, X_test, y_train, y_test),
        n_trials=n_trials,
        n_jobs=-1,
        show_progress_bar=True
    )

    # 최적의 하이퍼파라미터 출력
    print("\nBest trial:")
    trial = study.best_trial
    print("  Value (Negative MSE):", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 최적의 모델로 최종 평가
    best_params = study.best_params
    best_model = CatBoostRegressor(**best_params, random_seed=42, verbose=False)
    best_model.fit(X_train, y_train)

    # 테스트 세트에 대한 예측 및 MSE 계산
    y_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    print(f"\nTest MSE with best parameters: {test_mse:.4f}")

    # 학습 과정 시각화
    try:
        import plotly
        fig1 = optuna.visualization.plot_optimization_history(study)
        fig2 = optuna.visualization.plot_param_importances(study)
        fig1.show()
        fig2.show()
    except (ImportError, ModuleNotFoundError):
        print("Plotly is not installed. Skipping visualizations.")

    return best_params, test_mse, study

# 실행
best_params, test_mse, study = catboost_optuna_search(
    X_train, X_test, y_train, y_test, n_trials=100
)

ModuleNotFoundError: No module named 'optuna'

## Model (LGBM)

In [ ]:
!pip install lightgbm

In [ ]:
import optuna
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial, X_train, X_test, y_train, y_test):
    # 하이퍼파라미터 탐색 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5),

        # 추가적인 LightGBM 특화 파라미터
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.5),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.8, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.8, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7)
    }

    # 모델 생성
    lgbm_model = LGBMRegressor(**params,
                              random_state=42,
                              verbose=-1)  # verbose=-1로 설정하여 출력 억제

    # 교차 검증 점수 계산
    scores = cross_val_score(
        lgbm_model,
        X_train,
        y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=1  # 안정성을 위해 1로 설정
    )

    # MSE의 평균 반환
    return -scores.mean()

def lightgbm_optuna_search(X_train, X_test, y_train, y_test, n_trials=100):
    # Optuna study 생성
    study = optuna.create_study(direction='minimize')

    # 최적화 실행
    study.optimize(
        lambda trial: objective(trial, X_train, X_test, y_train, y_test),
        n_trials=n_trials,
        n_jobs=1,  # 안정성을 위해 1로 설정
        show_progress_bar=True
    )

    # 최적의 하이퍼파라미터 출력
    print("\nBest trial:")
    trial = study.best_trial
    print("  Value (Negative MSE):", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 최적의 모델로 최종 평가
    best_params = study.best_params
    best_model = LGBMRegressor(**best_params, random_state=42, verbose=-1)
    best_model.fit(X_train, y_train)

    # 테스트 세트에 대한 예측 및 MSE 계산
    y_pred = best_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    print(f"\nTest MSE with best parameters: {test_mse:.4f}")

    # 학습 과정 시각화
    try:
        optuna.visualization.plot_optimization_history(study)
        optuna.visualization.plot_param_importances(study)
        optuna.visualization.plot_param_correlations(study)
    except (ImportError, ModuleNotFoundError):
        print("Plotly is not installed. Skipping visualizations.")

    return best_params, test_mse, study

# 실행
best_params, test_mse, study = lightgbm_optuna_search(
    X_train, X_test, y_train, y_test, n_trials=100
)

## K-Fold MSE 비교

In [ ]:
!pip install lightgbm==3.3.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [ ]:
from lightgbm import LGBMRegressor

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
xgb_best_params = {
    'colsample_bytree': 0.9,
    'gamma': 0,
    'learning_rate': 0.01,
    'max_depth': 9,
    'min_child_weight': 5,
    'n_estimators': 1000,
    'subsample': 0.8
}


gbm_best_params = {
    'n_estimators': 800,
    'max_depth': 9,
    'learning_rate': 0.041755762142948585,
    'subsample': 0.9921333479894775,
    'min_samples_split': 7,
    'min_samples_leaf': 1,
    'max_features': 0.8356246524236172,
    'min_weight_fraction_leaf': 0.001390869498244622,
    'min_impurity_decrease': 0.07078251033719216,
    'validation_fraction': 0.2894972996229627,
    'n_iter_no_change': 15
}

cat_best_params = {
    'learning_rate': 0.09938584184224321,
    'depth': 8,
    'l2_leaf_reg': 1.538693189146234,
    'bootstrap_type': 'MVS',
    'random_strength': 1,
    'od_type': 'Iter',
    'od_wait': 17
}

lbgm_best_params = {
    'n_estimators': 800,
    'max_depth': 9,
    'learning_rate': 0.041755762142948585,
    'subsample': 0.9921333479894775,
    'min_samples_split': 7,
    'min_samples_leaf': 1,
    'max_features': 0.8356246524236172,
    'min_weight_fraction_leaf': 0.001390869498244622,
    'min_impurity_decrease': 0.07078251033719216,
    'validation_fraction': 0.2894972996229627,
    'n_iter_no_change': 15
}

In [ ]:
def kfold_cross_validation(X, y, model_type):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = []

    xgb_best_params = {
    'colsample_bytree': 0.9,
    'gamma': 0,
    'learning_rate': 0.01,
    'max_depth': 9,
    'min_child_weight': 5,
    'n_estimators': 1000,
    'subsample': 0.8
    }


    gbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.041755762142948585,
        'subsample': 0.9921333479894775,
        'min_samples_split': 7,
        'min_samples_leaf': 1,
        'max_features': 0.8356246524236172,
        'min_weight_fraction_leaf': 0.001390869498244622,
        'min_impurity_decrease': 0.07078251033719216,
        'validation_fraction': 0.2894972996229627,
        'n_iter_no_change': 15
    }

    cat_best_params = {
        'learning_rate': 0.09938584184224321,
        'depth': 8,
        'l2_leaf_reg': 1.538693189146234,
        'bootstrap_type': 'MVS',
        'random_strength': 1,
        'od_type': 'Iter',
        'od_wait': 17
    }

    lgbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.04,
        'num_leaves': 31,  # 추가
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'min_child_samples': 20,  # min_data_in_leaf와 동일
        'reg_alpha': 0.1,  # L1 정규화
        'reg_lambda': 0.1,  # L2 정규화
        'verbose': -1  # 경고 메시지 억제
    }

    if model_type == 'xgb':
        model = XGBRegressor(**xgb_best_params, random_state=42)
        model_name = 'XGBoost'
    elif model_type == 'gbm':
        model = GradientBoostingRegressor(**gbm_best_params, random_state=42)
        model_name = 'GradientBoosting'
    elif model_type == 'cat':
        model = CatBoostRegressor(**cat_best_params, random_state=42, verbose=False)
        model_name = 'CatBoost'
    elif model_type == 'lgbm':
        model = LGBMRegressor(**lgbm_best_params, random_state=42)
        model_name = 'LightGBM'

    print(f"\n{model_name} KFold Cross Validation Results:")

    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        X_train_fold = X.iloc[train_index]
        y_train_fold = y.iloc[train_index]
        X_val_fold = X.iloc[val_index]
        y_val_fold = y.iloc[val_index]

        # LightGBM의 경우 다른 방식으로 학습
        if model_type == 'lgbm':
            eval_set = [(X_val_fold, y_val_fold)]
            model.fit(
                X_train_fold,
                y_train_fold,
                eval_set=eval_set,
                eval_metric='mse',
                early_stopping_rounds=10,
                verbose=False
            )
        else:
            model.fit(X_train_fold, y_train_fold)

        y_pred_fold = model.predict(X_val_fold)
        mse_fold = mean_squared_error(y_val_fold, y_pred_fold)
        mse_scores.append(mse_fold)

        print(f"Fold {fold} MSE: {mse_fold:.4f}")

    mean_mse = np.mean(mse_scores)
    std_mse = np.std(mse_scores)
    print(f"\n{model_name} Average MSE: {mean_mse:.4f} (±{std_mse:.4f})")

    return mean_mse, std_mse

In [ ]:
kfold_cross_validation(X, y, 'xgb')


XGBoost KFold Cross Validation Results:
Fold 1 MSE: 1006.3449
Fold 2 MSE: 945.3873
Fold 3 MSE: 1069.1031
Fold 4 MSE: 1130.0779
Fold 5 MSE: 1077.7308

XGBoost Average MSE: 1045.7288 (±63.7256)


(1045.7288098203692, 63.72561130645175)

In [ ]:
kfold_cross_validation(X, y, 'gbm')


GradientBoosting KFold Cross Validation Results:
Fold 1 MSE: 1191.8898
Fold 2 MSE: 1033.9061
Fold 3 MSE: 1268.3832
Fold 4 MSE: 1170.0667
Fold 5 MSE: 1168.3542

GradientBoosting Average MSE: 1166.5200 (±75.6600)


(1166.520003454797, 75.65995509916576)

In [ ]:
kfold_cross_validation(X, y, 'cat')


CatBoost KFold Cross Validation Results:
Fold 1 MSE: 938.7849
Fold 2 MSE: 889.6603
Fold 3 MSE: 949.8359
Fold 4 MSE: 1080.4555
Fold 5 MSE: 1089.7539

CatBoost Average MSE: 989.6981 (±80.5441)


(989.6980840728866, 80.54411785926429)

In [ ]:
kfold_cross_validation(X, y, 'lgbm')


LightGBM KFold Cross Validation Results:


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Fold 1 MSE: 1068.6177


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Fold 2 MSE: 953.5273


...

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Fold 4 MSE: 1201.8832


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Fold 5 MSE: 1258.9586

LightGBM Average MSE: 1128.9974 (±107.4494)


(1128.9974187017933, 107.44943565611365)

## Ensemble

### StackingEnsemble

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold

In [ ]:
class StackingEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model
        self.base_predictions = None

    def fit(self, X, y):
        self.base_predictions = np.zeros((X.shape[0], len(self.base_models)))
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for i, model in enumerate(self.base_models):
            model_predictions = np.zeros(X.shape[0])

            for train_idx, val_idx in kf.split(X):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_val_fold = X.iloc[val_idx]

                model.fit(X_train_fold, y_train_fold)
                model_predictions[val_idx] = model.predict(X_val_fold)

            self.base_predictions[:, i] = model_predictions
            model.fit(X, y)

        self.meta_model.fit(self.base_predictions, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for model in self.base_models
        ])
        return self.meta_model.predict(meta_features)

In [ ]:
class WeightedBlending(BaseEstimator, RegressorMixin):
    def __init__(self, models, weights=None):
        self.models = models
        self.weights = weights
        self.fitted_weights = None

    def fit(self, X, y):
        # 각 모델 학습
        predictions = {}
        mse_scores = {}

        # K-fold 교차 검증으로 각 모델의 성능 평가
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for name, model in self.models.items():
            fold_predictions = np.zeros(len(y))

            for train_idx, val_idx in kf.split(X):
                X_train_fold = X.iloc[train_idx]
                y_train_fold = y.iloc[train_idx]
                X_val_fold = X.iloc[val_idx]
                y_val_fold = y.iloc[val_idx]

                # 모델 학습 및 예측
                model.fit(X_train_fold, y_train_fold)
                fold_predictions[val_idx] = model.predict(X_val_fold)

            # 각 모델의 MSE 계산
            mse = mean_squared_error(y, fold_predictions)
            mse_scores[name] = mse

            # 전체 데이터로 최종 학습
            model.fit(X, y)

        # 가중치 계산 (MSE의 역수 기반)
        if self.weights is None:
            mse_values = np.array(list(mse_scores.values()))
            weights = 1 / mse_values
            self.fitted_weights = weights / np.sum(weights)
        else:
            self.fitted_weights = self.weights

        return self

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))

        # 각 모델의 예측값 계산
        for i, model in enumerate(self.models.values()):
            predictions[:, i] = model.predict(X)

        # 가중치를 적용한 최종 예측값 계산
        weighted_predictions = np.average(predictions, weights=self.fitted_weights, axis=1)

        return weighted_predictions

    def get_weights(self):
        """학습된 가중치 반환"""
        return {name: weight for name, weight in zip(self.models.keys(), self.fitted_weights)}

In [ ]:
def create_and_evaluate_ensembles(X_train, X_test, y_train, y_test):
    # 모든 기본 모델 초기화
    xgb_best_params = {
        'colsample_bytree': 0.9,
        'gamma': 0,
        'learning_rate': 0.01,
        'max_depth': 9,
        'min_child_weight': 5,
        'n_estimators': 1000,
        'subsample': 0.8
    }


    gbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.041755762142948585,
        'subsample': 0.9921333479894775,
        'min_samples_split': 7,
        'min_samples_leaf': 1,
        'max_features': 0.8356246524236172,
        'min_weight_fraction_leaf': 0.001390869498244622,
        'min_impurity_decrease': 0.07078251033719216,
        'validation_fraction': 0.2894972996229627,
        'n_iter_no_change': 15
    }

    cat_best_params = {
        'learning_rate': 0.09938584184224321,
        'depth': 8,
        'l2_leaf_reg': 1.538693189146234,
        'bootstrap_type': 'MVS',
        'random_strength': 1,
        'od_type': 'Iter',
        'od_wait': 17
    }

    lgbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.04,
        'num_leaves': 31,  # 추가
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'min_child_samples': 20,  # min_data_in_leaf와 동일
        'reg_alpha': 0.1,  # L1 정규화
        'reg_lambda': 0.1,  # L2 정규화
        'verbose': -1  # 경고 메시지 억제
    }


    models = {
        'xgb': XGBRegressor(**xgb_best_params, random_state=42),
        'gbm': GradientBoostingRegressor(**gbm_best_params, random_state=42),
        'cat': CatBoostRegressor(**cat_best_params, random_state=42, verbose=False),
        'lgbm': LGBMRegressor(**lgbm_best_params, random_state=42)
    }

    # 개별 모델 평가
    print("=== Individual Model Performance ===")
    individual_predictions = {}
    individual_mse = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        mse = mean_squared_error(y_test, pred)
        individual_predictions[name] = pred
        individual_mse[name] = mse
        print(f"{name} MSE: {mse:.4f}")

    # Stacking Ensemble
    print("\n=== Stacking Ensemble ===")
    base_models = list(models.values())
    meta_model = XGBRegressor(random_state=42)

    stacking = StackingEnsemble(base_models, meta_model)
    stacking.fit(X_train, y_train)
    stacking_pred = stacking.predict(X_test)
    stacking_mse = mean_squared_error(y_test, stacking_pred)
    print(f"Stacking Ensemble MSE: {stacking_mse:.4f}")

    # Weighted Blending
    print("\n=== Weighted Blending ===")
    blending = WeightedBlending(models)
    blending.fit(X_train, y_train)
    blending_pred = blending.predict(X_test)
    blending_mse = mean_squared_error(y_test, blending_pred)

    # 학습된 가중치 출력
    weights = blending.get_weights()
    print("\nLearned Weights:")
    for name, weight in weights.items():
        print(f"{name}: {weight:.4f}")

    print(f"\nWeighted Blending MSE: {blending_mse:.4f}")

    # 결과 비교
    print("\n=== Final Comparison ===")
    all_results = {
        **individual_mse,
        'Stacking_Ensemble': stacking_mse,
        'Weighted_Blending': blending_mse
    }

    for name, mse in all_results.items():
        print(f"{name}: {mse:.4f}")

    return {
        'individual_mse': individual_mse,
        'stacking_mse': stacking_mse,
        'blending_mse': blending_mse,
        'predictions': {
            'individual': individual_predictions,
            'stacking': stacking_pred,
            'blending': blending_pred
        },
        'blending_weights': weights
    }

# 실행
results = create_and_evaluate_ensembles(X_train, X_test, y_train, y_test)

=== Individual Model Performance ===
xgb MSE: 1010.5406
gbm MSE: 1135.3812
cat MSE: 948.2036
lgbm MSE: 1065.8756

=== Stacking Ensemble ===
Stacking Ensemble MSE: 1154.0166

=== Weighted Blending ===

Learned Weights:
xgb: 0.2556
gbm: 0.2248
cat: 0.2651
lgbm: 0.2545

Weighted Blending MSE: 964.1343

=== Final Comparison ===
xgb: 1010.5406
gbm: 1135.3812
cat: 948.2036
lgbm: 1065.8756
Stacking_Ensemble: 1154.0166
Weighted_Blending: 964.1343


In [ ]:
# CatBoost vs. Weight Blending (KFold 평균 비교)

def create_and_evaluate_ensembles(X, y, test_size=0.2):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # 모델 파라미터 설정
    xgb_best_params = {
        'colsample_bytree': 0.9,
        'gamma': 0,
        'learning_rate': 0.01,
        'max_depth': 9,
        'min_child_weight': 5,
        'n_estimators': 1000,
        'subsample': 0.8
    }

    gbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.041755762142948585,
        'subsample': 0.9921333479894775,
        'min_samples_split': 7,
        'min_samples_leaf': 1,
        'max_features': 0.8356246524236172,
        'min_weight_fraction_leaf': 0.001390869498244622,
        'min_impurity_decrease': 0.07078251033719216,
        'validation_fraction': 0.2894972996229627,
        'n_iter_no_change': 15
    }

    cat_best_params = {
        'learning_rate': 0.09938584184224321,
        'depth': 8,
        'l2_leaf_reg': 1.538693189146234,
        'bootstrap_type': 'MVS',
        'random_strength': 1,
        'od_type': 'Iter',
        'od_wait': 17
    }

    lgbm_best_params = {
        'n_estimators': 800,
        'max_depth': 9,
        'learning_rate': 0.04,
        'num_leaves': 31,
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'min_child_samples': 20,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'verbose': -1
    }

    # K-fold 설정
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # MSE 저장을 위한 딕셔너리
    model_mse_scores = {
        'catboost': [],
        'blending': []
    }

    # K-fold 교차 검증 수행
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        print(f"\n=== Fold {fold} ===")

        # 데이터 분할
        X_fold_train = X_train.iloc[train_idx]
        y_fold_train = y_train.iloc[train_idx]
        X_fold_val = X_train.iloc[val_idx]
        y_fold_val = y_train.iloc[val_idx]

        # CatBoost 단일 모델
        catboost = CatBoostRegressor(**cat_best_params, random_state=42, verbose=False)
        catboost.fit(X_fold_train, y_fold_train)
        cat_pred = catboost.predict(X_fold_val)
        cat_mse = mean_squared_error(y_fold_val, cat_pred)
        model_mse_scores['catboost'].append(cat_mse)
        print(f"CatBoost MSE: {cat_mse:.4f}")

        # Weighted Blending을 위한 모든 모델
        models = {
            'xgb': XGBRegressor(**xgb_best_params, random_state=42),
            'gbm': GradientBoostingRegressor(**gbm_best_params, random_state=42),
            'cat': CatBoostRegressor(**cat_best_params, random_state=42, verbose=False),
            'lgbm': LGBMRegressor(**lgbm_best_params, random_state=42)
        }

        # Weighted Blending
        blending = WeightedBlending(models)
        blending.fit(X_fold_train, y_fold_train)
        blending_pred = blending.predict(X_fold_val)
        blending_mse = mean_squared_error(y_fold_val, blending_pred)
        model_mse_scores['blending'].append(blending_mse)
        print(f"Weighted Blending MSE: {blending_mse:.4f}")

    # 최종 결과 출력
    print("\n=== Final Results (Average over 5 folds) ===")
    final_results = {}
    for name, scores in model_mse_scores.items():
        mean_mse = np.mean(scores)
        std_mse = np.std(scores)
        final_results[name] = {
            'mean_mse': mean_mse,
            'std_mse': std_mse
        }
        print(f"{name}: MSE = {mean_mse:.4f} (±{std_mse:.4f})")

    # 테스트 세트에 대한 최종 평가
    print("\n=== Test Set Final Evaluation ===")

    # CatBoost 단일 모델 평가
    final_catboost = CatBoostRegressor(**cat_best_params, random_state=42, verbose=False)
    final_catboost.fit(X_train, y_train)
    test_cat_pred = final_catboost.predict(X_test)
    test_cat_mse = mean_squared_error(y_test, test_cat_pred)
    print(f"CatBoost Test MSE: {test_cat_mse:.4f}")

    # 최종 Weighted Blending 모델
    final_models = {
        'xgb': XGBRegressor(**xgb_best_params, random_state=42),
        'gbm': GradientBoostingRegressor(**gbm_best_params, random_state=42),
        'cat': CatBoostRegressor(**cat_best_params, random_state=42, verbose=False),
        'lgbm': LGBMRegressor(**lgbm_best_params, random_state=42)
    }

    # Blending 테스트 평가
    blending = WeightedBlending(final_models)
    blending.fit(X_train, y_train)
    test_blending_pred = blending.predict(X_test)
    test_blending_mse = mean_squared_error(y_test, test_blending_pred)
    print(f"Blending Test MSE: {test_blending_mse:.4f}")

    # 학습된 가중치 출력
    print("\nLearned Blending Weights:")
    weights = blending.get_weights()
    for name, weight in weights.items():
        print(f"{name}: {weight:.4f}")

    return {
        'cross_validation_results': final_results,
        'test_mse': {
            'catboost': test_cat_mse,
            'blending': test_blending_mse
        },
        'test_predictions': {
            'catboost': test_cat_pred,
            'blending': test_blending_pred
        },
        'blending_weights': weights
    }

# 실행
results = create_and_evaluate_ensembles(X, y)


=== Fold 1 ===
CatBoost MSE: 1043.0085
Weighted Blending MSE: 1031.6728

=== Fold 2 ===
CatBoost MSE: 1065.6834
Weighted Blending MSE: 1098.0300

=== Fold 3 ===
CatBoost MSE: 1143.3213
Weighted Blending MSE: 1133.6127

=== Fold 4 ===
CatBoost MSE: 958.9866
Weighted Blending MSE: 940.1889

=== Fold 5 ===
CatBoost MSE: 1182.9396
Weighted Blending MSE: 1100.2460

=== Final Results (Average over 5 folds) ===
catboost: MSE = 1078.7879 (±78.5065)
blending: MSE = 1060.7501 (±68.7593)

=== Test Set Final Evaluation ===
CatBoost Test MSE: 948.2036
Blending Test MSE: 964.1343

Learned Blending Weights:
xgb: 0.2556
gbm: 0.2248
cat: 0.2651
lgbm: 0.2545


## 최종 test data prediction

In [ ]:
df_test = pd.read_csv('./question1-test_cleaned.csv')

df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,hour,dayofweek
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,2012,4,13,0
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,2012,4,14,0
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,2012,4,15,0
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,2012,4,16,0
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,2012,4,17,0


In [ ]:
df_train = pd.read_csv('./question1-train_cleaned.csv')

df_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,hour,dayofweek
0,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,1,2011,1,5,5
1,2011-01-01 10:00:00,1,0,0,1,15.58,19.695,76,16.9979,36,2011,1,10,5
2,2011-01-01 11:00:00,1,0,0,1,14.76,16.665,81,19.0012,56,2011,1,11,5
3,2011-01-01 12:00:00,1,0,0,1,17.22,21.210,77,19.0012,84,2011,1,12,5
4,2011-01-01 13:00:00,1,0,0,2,18.86,22.725,72,19.9995,94,2011,1,13,5


In [ ]:
categorical_col = ['workingday', 'weather','dayofweek','month','year','hour']
numerical_col = ['temp', 'humidity']

target_col = "count"

X_train = df_train[categorical_col + numerical_col]
y_train = df_train[target_col]

X_test = df_test[categorical_col + numerical_col]
y_test = df_test[target_col]

In [ ]:
cat_best_params = {
    'learning_rate': 0.01,
    'depth': 8,
    'l2_leaf_reg': 1.538693189146234,
    'bootstrap_type': 'MVS',
    'random_strength': 1,
    'od_type': 'Iter',
    'od_wait': 17
}

model = CatBoostRegressor(**cat_best_params, random_state=24, verbose=False)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(y_pred)

[213.948018   211.25172676 228.71975272 ... 126.7461037   85.86096202
  58.99867682]


In [ ]:
df_test['count'] = y_pred

df_test.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,hour,dayofweek
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,232.488555,2012,4,13,0
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,208.719464,2012,4,14,0
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,236.461515,2012,4,15,0
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,332.590573,2012,4,16,0
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,565.610244,2012,4,17,0
5,2012-04-02 18:00:00,2,0,1,1,20.50,24.240,27,16.9979,534.480324,2012,4,18,0
6,2012-04-02 19:00:00,2,0,1,1,20.50,24.240,27,16.9979,399.097062,2012,4,19,0
7,2012-04-02 20:00:00,2,0,1,1,18.86,22.725,31,11.0014,282.310017,2012,4,20,0
8,2012-04-02 21:00:00,2,0,1,1,18.86,22.725,28,8.9981,182.283610,2012,4,21,0
9,2012-04-02 22:00:00,2,0,1,1,17.22,21.210,44,12.9980,133.267087,2012,4,22,0


## Submission

In [ ]:
submission = pd.read_excel('./question1-test - features.xlsx')

In [ ]:
submission.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,NaN,NaN
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,NaN,NaN
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,NaN,NaN
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,NaN,NaN
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,NaN,NaN
5,2012-04-02 18:00:00,2,0,1,1,20.50,24.240,27,16.9979,NaN,NaN,NaN
6,2012-04-02 19:00:00,2,0,1,1,20.50,24.240,27,16.9979,NaN,NaN,NaN
7,2012-04-02 20:00:00,2,0,1,1,18.86,22.725,31,11.0014,NaN,NaN,NaN
8,2012-04-02 21:00:00,2,0,1,1,18.86,22.725,28,8.9981,NaN,NaN,NaN
9,2012-04-02 22:00:00,2,0,1,1,17.22,21.210,44,12.9980,NaN,NaN,NaN


In [ ]:
submission['count'] = y_pred

submission.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,NaN,213.948018
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,NaN,211.251727
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,NaN,228.719753
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,NaN,336.282901
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,NaN,538.132626
5,2012-04-02 18:00:00,2,0,1,1,20.50,24.240,27,16.9979,NaN,NaN,517.145296
6,2012-04-02 19:00:00,2,0,1,1,20.50,24.240,27,16.9979,NaN,NaN,407.445587
7,2012-04-02 20:00:00,2,0,1,1,18.86,22.725,31,11.0014,NaN,NaN,279.634377
8,2012-04-02 21:00:00,2,0,1,1,18.86,22.725,28,8.9981,NaN,NaN,205.502837
9,2012-04-02 22:00:00,2,0,1,1,17.22,21.210,44,12.9980,NaN,NaN,137.436910


In [ ]:
submission

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2012-04-02 13:00:00,2,0,1,1,18.86,22.725,36,19.9995,NaN,NaN,213.948018
1,2012-04-02 14:00:00,2,0,1,1,18.86,22.725,31,27.9993,NaN,NaN,211.251727
2,2012-04-02 15:00:00,2,0,1,1,19.68,23.485,33,27.9993,NaN,NaN,228.719753
3,2012-04-02 16:00:00,2,0,1,1,20.50,24.240,34,22.0028,NaN,NaN,336.282901
4,2012-04-02 17:00:00,2,0,1,1,20.50,24.240,29,19.0012,NaN,NaN,538.132626
...,...,...,...,...,...,...,...,...,...,...,...,...
1856,2012-08-03 22:00:00,3,0,1,1,31.16,36.365,70,16.9979,NaN,NaN,260.200874
1857,2012-08-03 23:00:00,3,0,1,1,30.34,35.605,79,8.9981,NaN,NaN,195.575178
1858,2012-08-04 00:00:00,3,0,0,1,30.34,35.605,79,11.0014,NaN,NaN,126.746104
1859,2012-08-04 01:00:00,3,0,0,1,29.52,35.605,84,12.9980,NaN,NaN,85.860962


In [ ]:
submission.to_csv('question1-submission_final.csv', index=False)

In [ ]:
categorical_col = ['workingday', 'weather','dayofweek','month','year','hour']
numerical_col = ['temp', 'humidity']

target_col = "count"

X = df[categorical_col + numerical_col]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
cat_best_params = {
    'learning_rate': 0.09938584184224321,
    'depth': 8,
    'l2_leaf_reg': 1.538693189146234,
    'bootstrap_type': 'MVS',
    'random_strength': 1,
    'od_type': 'Iter',
    'od_wait': 17
}

model = CatBoostRegressor(**cat_best_params, random_state=42, verbose=False)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 948.2036199210638


In [ ]:
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
comparison_df = comparison_df.reset_index(drop=True)

comparison_df

,Actual,Predicted
0,158,138.699561
1,8,-17.876013
2,143,141.583868
3,14,10.775359
4,4,17.689301
...,...,...
1774,165,170.718126
1775,114,126.740846
1776,234,265.783028
1777,129,143.521782
